In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
!pip install "numpy<2"
!pip install pyensembl
!pip install scanpy
!pip install torch
!pip install pandas

ERROR: Operation cancelled by user
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 11.1 MB/s eta 0:00:00
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4186 sha256=0176db940572aa86d5d4c341fbe6eb692bba4bc96f3dd57ecedeeffb3c5

In [3]:
from torch.utils.data import random_split, DataLoader, Dataset
from IPython.display import clear_output, display

In [ ]:
!pip install causal-conv1d --no-build-isolation
!pip install "mamba-ssm @ git+https://github.com/state-spaces/mamba.git#egg=mamba-ssm"



  Cloning https://github.com/state-spaces/mamba.git to /tmp/pip-install-gwao30_3/mamba-ssm_0c6c8584646f4f90a181c85e3c938aa0
  Running command git clone --filter=blob:none --quiet https://github.com/state-spaces/mamba.git /tmp/pip-install-gwao30_3/mamba-ssm_0c6c8584646f4f90a181c85e3c938aa0
  Resolved https://github.com/state-spaces/mamba.git to commit 10b5d6358f27966f6a40e4bf0baa17a460688128
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import scanpy as sc
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pyensembl
import time

/usr/local/lib/python3.12/dist-packages/scanpy/_utils/__init__.py:33: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version
/usr/local/lib/python3.12/dist-packages/scanpy/__init__.py:24: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):
/usr/local/lib/python3.12/dist-packages/scanpy/readwrite.py:16: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  if Version(anndata.__version__) >= Version("0.11.0rc2"):


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/MambaPerturb/virtual_cell')

In [ ]:
import src.model as m
import src.data_cleaning as dc
import src.position_encoding as pos
import src.pathway_encoding as path
from src.model import TranscriptomePredictor

In [ ]:
ensembl_data = pyensembl.EnsemblRelease(109)
ensembl_data.download()
ensembl_data.index()

In [ ]:
adata = sc.read_h5ad('/content/drive/MyDrive/MambaPerturb/virtual_cell/vcc_data/adata_Training.h5ad')
#adata.obs['perturbation_idx'] = np.random.randint(0, CONFIG["n_perturbations"], size=adata.n_obs)
gene_names = pd.read_csv('/content/drive/MyDrive/MambaPerturb/virtual_cell/vcc_data/gene_names.csv', header = None)
gene_names = gene_names.iloc[:, 0].tolist()


CONFIG = {
    "n_genes": len(gene_names),
    "n_perturbations": len(adata.obs['target_gene'].unique().tolist()),
    "n_chromosomes": 25,

    "perturbation_dim": 256,      # Condition embedding
    "chrom_embedding_dim": 16,     # Learned in-model for chromosome identity
    "locus_fourier_features": 8,   # Number of Fourier frequency pairs (2*F)
    "pathway_dim": 50,             # From pre-trained Autoencoder(based on hallmark MSigDB)
    "gene_identity_dim": 189,       # Main learnable gene embedding

    # Backbone dims
    "d_model": 512,                # Mamba hidden size
    "mamba_layers": 4,

    # Head
    "prediction_head": "probabilistic", # "linear" | "probabilistic"

    # Training
    "batch_size": 64,
    "learning_rate": 5e-5, # Lowered LR for AdamW stability
    "epochs": 100,
}

# Derived dimensions for clarity
# Positional encoding dim = chromosome embedding + MLP output from Fourier features
POS_DIM = CONFIG["chrom_embedding_dim"] + CONFIG["chrom_embedding_dim"]
GENE_FEAT_DIM = CONFIG["gene_identity_dim"] + CONFIG["pathway_dim"] + POS_DIM

In [ ]:

unique_perturbations = adata.obs['target_gene'].unique().tolist()
perturbation_to_idx_map = {name: i for i, name in enumerate(unique_perturbations)}
adata.obs['perturbation_idx'] = adata.obs['target_gene'].map(perturbation_to_idx_map)
control_adata = dc.get_control_data(adata)
chr_idx, locus_norm, locus_fourier = pos.precompute_positional_indices(ensembl_data, gene_names, CONFIG)




--- Preparing positional indices using pyensembl ---


Fetching gene positions: 100%|██████████| 18080/18080 [00:01<00:00, 16805.31it/s]


SUCCESS: Generated positional tensors with shapes:
Chromosome Indices (chr_idx): torch.Size([18080])
Normalized Locus (locus_norm): torch.Size([18080, 1])
Locus Fourier Features (locus_fourier): torch.Size([18080, 16])


In [ ]:
    # Pre-compute Fixed Features (run once, then save/load)
#filtered_data = dc.clean_and_preprocess_data(adata)
    # find dataset that can represent a nnoramla scell set
pathway_feats = path.precompute_pathway_features(control_adata, CONFIG)

--- Precomputing pathway features on control data ---
AE epoch 10/50 | recon MSE: 23.9425
AE epoch 20/50 | recon MSE: 9.8262
AE epoch 30/50 | recon MSE: 7.6511
AE epoch 40/50 | recon MSE: 7.2286
AE epoch 50/50 | recon MSE: 7.0843
Generated pathway_features shape: torch.Size([18080, 50])


In [ ]:
len(gene_names)

18080

In [ ]:
    # Instantiate Model, Dataset, Dataloader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")
model = TranscriptomePredictor(CONFIG, GENE_FEAT_DIM, pathway_feats, chr_idx, locus_fourier).to(device)

dataset = m.PerturbationDataset(adata, CONFIG) # Pass CONFIG to the dataset constructor
train_loader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True)


#train_size = int(0.8 * len(dataset))
#val_size = len(dataset) - train_size

# 2. Perform the split
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 3. Create separate DataLoaders for each set
#train_loader = DataLoader(train_dataset, batch_size=CONFIG["batch_size"], shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=CONFIG["batch_size"], shuffle=False)

#print(f"Data split into {len(train_dataset)} training samples and {len(val_dataset)} validation samples.")


Using device: cuda
Dataset using log-normalized data from adata.X for linear head.


In [ ]:
'''device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")
model = TranscriptomePredictor(CONFIG, GENE_FEAT_DIM, pathway_feats, chr_idx, locus_fourier).to(device)

dataset = m.PerturbationDataset(adata, CONFIG) # Pass CONFIG to the dataset constructor
dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True)'''

    # Train
loss_history = m.train_model(model, train_loader, CONFIG)


--- Starting Model Training ---


RuntimeError: Triton Error [CUDA]: an illegal memory access was encountered

In [ ]:
print("\n--- Starting Model Training & Validation ---")
opt = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-2)

    # --- 1. Initialization for Plotting ---
train_losses = []
val_losses = []

    # Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(10, 6))
display(fig) # Initially display the empty figure

for ep in range(CONFIG["epochs"]):
        # --- Training Phase ---
    model.train()
    total_train_loss = 0.0
    pbar_train = tqdm(train_loader, desc=f"Epoch {ep+1}/{CONFIG['epochs']} [Train]", leave=False)
    for batch in pbar_train:
        opt.zero_grad(set_to_none=True)
        pert_idx = batch["perturbation_idx"].to(next(model.parameters()).device)
        targets = batch["target_expression"].to(next(model.parameters()).device)

        if CONFIG["prediction_head"] == "probabilistic":
            mean_log, disp_log = model(pert_idx)
            loss = model.nb_nll_loss(targets, mean_log, disp_log)
        else:
            predictions = model(pert_idx)
            loss = nn.functional.mse_loss(predictions, targets)

        loss.backward()
        opt.step()
        total_train_loss += loss.item()
        pbar_train.set_postfix({"Loss": f"{loss.item():.4f}"})

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

        # --- Validation Phase ---
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        pbar_val = tqdm(train_loader, desc=f"Epoch {ep+1}/{CONFIG['epochs']} [Val]", leave=False)
        for batch in pbar_val:
            pert_idx = batch["perturbation_idx"].to(next(model.parameters()).device)
            targets = batch["target_expression"].to(next(model.parameters()).device)

            if CONFIG["prediction_head"] == "probabilistic":
                mean_log, disp_log = model(pert_idx)
                loss = model.nb_nll_loss(targets, mean_log, disp_log)
            else:
                predictions = model(pert_idx)
                loss = nn.functional.mse_loss(predictions, targets)

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

        # --- 2. Live Plotting Logic ---
    ax.clear()
    ax.plot(train_losses, label='Training Loss', marker='o')
    ax.plot(val_losses, label='Validation Loss', marker='o')
    ax.set_title("Training & Validation Loss vs. Epoch")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Average Loss")
    ax.legend()
    ax.grid(True)
    ax.set_xticks(range(len(train_losses)))
    ax.set_xticklabels(range(1, len(train_losses) + 1))


    status_text = f"Epoch {ep+1}: Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}"
    fig.suptitle(status_text, y=0.02, fontsize=12) # Use suptitle for better placement

    fig.tight_layout(rect=[0, 0.05, 1, 1]) # Adjust layout to make room for the text

    clear_output(wait=True)
    display(fig)
    time.sleep(0.1) # Small pause for smooth rendering

    print(f"Epoch {ep+1}/{CONFIG['epochs']} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

    # --- 3. Cleanup ---
plt.close(fig)
print("\n--- Training Complete ---")